In [1]:
import sys
from pathlib import Path
import pandas as pd

# Notebook cell
%load_ext autoreload
%autoreload 2

# Get root directory (assuming notebook is in root/notebooks/)
NOTEBOOK_DIR = Path.cwd()
ROOT_DIR = NOTEBOOK_DIR.parent if NOTEBOOK_DIR.name == 'notebooks' else NOTEBOOK_DIR

# Add src directory to Python path
sys.path.append(str(ROOT_DIR / 'src'))

# Verify path
print(f"Python will look in these locations:\n{sys.path}")


# --- Execute the processor ---
import utils
from config import date_str, DOWNLOAD_DIR, DEST_DIR

path_data = f'..\data\{date_str}_df_finviz_merged.parquet'
# path_corr = '..\data\df_corr_emv_matrix.parquet'
# path_cov = '..\data\df_cov_emv_matrix.parquet'
# path_output = f'..\picks\{date_str}_portf.txt'


print(utils.__file__)  # Should point to your src/utils.py
print(f'path_date: {path_data}')
# print(f'path_corr: {path_corr}')
# print(f'path_cov: {path_cov}')  
# print(f'path_output: {path_output}')
# print((f'date_str: {date_str}'))



Python will look in these locations:
['C:\\Users\\ping\\.pyenv\\pyenv-win\\versions\\3.11.9\\python311.zip', 'C:\\Users\\ping\\.pyenv\\pyenv-win\\versions\\3.11.9\\DLLs', 'C:\\Users\\ping\\.pyenv\\pyenv-win\\versions\\3.11.9\\Lib', 'C:\\Users\\ping\\.pyenv\\pyenv-win\\versions\\3.11.9', 'C:\\Users\\ping\\Files_win10\\python\\py311\\.venv', '', 'C:\\Users\\ping\\Files_win10\\python\\py311\\.venv\\Lib\\site-packages', 'C:\\Users\\ping\\Files_win10\\python\\py311\\.venv\\Lib\\site-packages\\win32', 'C:\\Users\\ping\\Files_win10\\python\\py311\\.venv\\Lib\\site-packages\\win32\\lib', 'C:\\Users\\ping\\Files_win10\\python\\py311\\.venv\\Lib\\site-packages\\Pythonwin', 'c:\\Users\\ping\\Files_win10\\python\\py311\\stocks\\src']


c:\Users\ping\Files_win10\python\py311\stocks\src\utils.py
path_date: ..\data\2025-04-23_df_finviz_merged.parquet


In [2]:
import pandas as pd
# Set pandas display options to show more columns and rows
pd.set_option('display.max_columns', None)  # Show all columns
# pd.set_option('display.max_rows', 10)       # Limit to 10 rows for readability
# pd.set_option('display.width', None)        # Let the display adjust to the window
# pd.set_option('display.max_colwidth', None) # Show full content of each cell
pd.set_option('display.max_rows', 100)
# pd.set_option('display.width', 120)
pd.set_option('display.float_format', '{:.4f}'.format)

In [3]:
import pandas as pd

def get_column_values_above_threshold(df, column_name='Avg Volume, M', threshold=0.75):
  """
  Analyzes the number and percentage of values in a DataFrame column that are above a specified threshold,
  and returns the filtered DataFrame.

  Args:
    df (pd.DataFrame): The input DataFrame.
    column_name (str): The name of the column to analyze. Defaults to 'Avg Volume, M'.
    threshold (float): The threshold value to compare against. Defaults to 1.00.

  Returns:
    pd.DataFrame: A DataFrame containing only the rows where the specified column's value is above the threshold.
  """
  
  count_before = len(df)
  above_threshold_df = df[df[column_name] > threshold]
  count_after = len(above_threshold_df)
  percentage = (count_after / len(df)) * 100

  print(f"count_before: {count_before}")
  print(f"count_after above threshold ({threshold}): {count_after}")
  print(f"Percentage above threshold ({threshold}): {percentage:.2f}%")

  return above_threshold_df


In [4]:
df_data = pd.read_parquet(path_data)

# liquidity filter, Avg Volume, M > 0.75M
df_data = get_column_values_above_threshold(df_data, column_name='Avg Volume, M', threshold=0.75)

# Drop specified columns with NaNs in df_data
df_data = df_data.drop(['All-Time High %', 'All-Time Low %', 'Dividend %'], axis=1)

# df_corr = pd.read_parquet(path_corr)
# df_cov = pd.read_parquet(path_cov)

# print(f'\ndf_cov.shape: {df_cov.shape}')
# display(df_cov.head())

# print(f'\ndf_corr.shape: {df_corr.shape}')
# display(df_corr.head())

print(f'\ndf_data.shape: {df_data.shape}')
display(df_data.head())
display((df_data.describe()))

count_before: 1537
count_after above threshold (0.75): 1202
Percentage above threshold (0.75): 78.20%

df_data.shape: (1202, 52)


,Company,Info,"MktCap AUM, M",Beta,RSI,Perf YTD %,Perf 3D %,Perf Week %,Perf Month %,Perf Quart %,Perf Half %,Perf Year %,SMA20 %,SMA50 %,SMA200 %,50D High %,50D Low %,52W High %,52W Low %,ATR,ATR/Price %,Volatility W %,Volatility M %,"Volume, M","Avg Volume, M",Rel Volume,Price,Change %,Sharpe 3d,Sortino 3d,Omega 3d,Sharpe 5d,Sortino 5d,Omega 5d,Sharpe 10d,Sortino 10d,Omega 10d,Sharpe 15d,Sortino 15d,Omega 15d,Sharpe 30d,Sortino 30d,Omega 30d,Sharpe 60d,Sortino 60d,Omega 60d,Sharpe 120d,Sortino 120d,Omega 120d,Sharpe 250d,Sortino 250d,Omega 250d
Ticker,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,
AAPL,Apple Inc,"Technology, Consumer Electronics",3073520.0000,1.2900,49.0400,-18.3000,3.8684,1.2200,-7.3100,-8.5900,-12.9400,24.0000,0.8200,-7.0000,-10.1900,-18.1600,20.9100,-21.3400,24.0600,9.9900,4.8827,2.6900,4.8400,52.0936,61.3900,0.8500,204.6000,2.4300,66.9795,8757.8179,781.2084,8.9293,21.2395,3.6759,1.7656,2.6850,1.3000,-1.2684,-1.9802,0.8076,-0.4748,-0.7227,0.9142,-1.0764,-1.5500,0.8079,-0.6064,-0.8638,0.8847,0.6438,0.9567,1.1350
MSFT,Microsoft Corporation,"Technology, Software - Infrastructure",2783210.0000,0.9800,47.3400,-11.1800,1.7973,-2.9400,-4.7500,-16.0900,-10.4700,-6.2000,-0.4000,-3.5200,-10.0200,-10.7100,8.5800,-20.0600,8.5800,12.8700,3.4376,2.4800,3.3600,20.2058,25.2100,0.8000,374.3900,2.0600,582.6925,8757.8179,781.2084,1.3341,2.3249,1.2221,-3.4440,-4.3079,0.5914,-0.4842,-0.9400,0.9150,-0.3997,-0.7150,0.9265,-2.1398,-3.0785,0.6614,-0.9786,-1.3746,0.8308,-0.3687,-0.5088,0.9352
NVDA,NVIDIA Corp,"Technology, Semiconductors",2506120.0000,2.1100,44.5900,-23.5200,1.2021,-8.4600,-15.4000,-30.1600,-25.5700,34.7900,-2.8300,-11.5900,-18.3400,-28.4000,18.5800,-32.9300,31.3000,7.1100,6.9224,4.1600,6.3600,245.3014,313.7900,0.7800,102.7100,3.8600,36.2350,8757.8179,781.2084,-1.5428,-2.2780,0.7922,-4.4274,-5.0789,0.5044,-0.7381,-1.2607,0.8786,-0.9640,-1.5560,0.8356,-1.0929,-1.5654,0.8281,-0.8050,-1.0933,0.8679,0.6615,0.9583,1.1193
AMZN,Amazon.com Inc,"Consumer Cyclical, Internet Retail",1916590.0000,1.3100,46.2300,-17.6800,4.6289,0.5600,-11.1500,-23.1500,-4.4400,3.4200,-1.2000,-8.9100,-9.4000,-22.7900,11.9100,-25.5300,19.1200,9.7100,5.3765,3.2800,5.0400,61.8203,48.8300,1.2700,180.6000,4.2800,111.2769,8757.8179,781.2084,4.1218,8.9966,1.8993,-2.9677,-4.0059,0.6450,-1.5029,-2.2917,0.7822,-1.1801,-1.7341,0.8164,-2.4154,-3.2714,0.6553,-0.2043,-0.2953,0.9656,0.1170,0.1650,1.0206
GOOGL,Alphabet Inc,"Communication Services, Internet Content & Inf...",1907420.0000,1.0000,47.6800,-17.9300,2.7719,-0.6100,-7.3500,-21.6900,-4.9400,0.8200,0.8400,-5.9300,-9.9800,-17.4500,10.5500,-24.9700,10.5500,5.9900,3.8558,2.8600,3.7700,30.3598,35.0200,0.8700,155.3500,2.5600,4878.4214,8757.8179,781.2084,2.0699,3.9174,1.3587,-1.4315,-2.0259,0.8181,-0.1410,-0.2536,0.9772,-1.2256,-1.8601,0.8185,-2.2889,-3.0174,0.6820,-0.4243,-0.6060,0.9316,-0.0291,-0.0418,0.9949


,"MktCap AUM, M",Beta,RSI,Perf YTD %,Perf 3D %,Perf Week %,Perf Month %,Perf Quart %,Perf Half %,Perf Year %,SMA20 %,SMA50 %,SMA200 %,50D High %,50D Low %,52W High %,52W Low %,ATR,ATR/Price %,Volatility W %,Volatility M %,"Volume, M","Avg Volume, M",Rel Volume,Price,Change %,Sharpe 3d,Sortino 3d,Omega 3d,Sharpe 5d,Sortino 5d,Omega 5d,Sharpe 10d,Sortino 10d,Omega 10d,Sharpe 15d,Sortino 15d,Omega 15d,Sharpe 30d,Sortino 30d,Omega 30d,Sharpe 60d,Sortino 60d,Omega 60d,Sharpe 120d,Sortino 120d,Omega 120d,Sharpe 250d,Sortino 250d,Omega 250d
count,1202.0000,1202.0000,1202.0000,1202.0000,1202.0000,1202.0000,1202.0000,1202.0000,1202.0000,1202.0000,1202.0000,1202.0000,1202.0000,1202.0000,1202.0000,1202.0000,1202.0000,1202.0000,1202.0000,1202.0000,1202.0000,1202.0000,1202.0000,1202.0000,1202.0000,1202.0000,1202.0000,1202.0000,1202.0000,1202.0000,1202.0000,1202.0000,1202.0000,1202.0000,1202.0000,1202.0000,1202.0000,1202.0000,1202.0000,1202.0000,1202.0000,1202.0000,1202.0000,1202.0000,1202.0000,1202.0000,1202.0000,1202.0000,1202.0000,1202.0000
mean,58322.0715,0.9930,49.7131,-3.7685,1.8049,1.3888,-5.8134,-7.4467,-5.3862,12.3486,0.5539,-3.7544,-4.4720,-14.7292,14.5073,-20.8245,30.3075,4.1171,4.0901,2.7083,4.0342,7.2051,6.6729,1.0309,99.2806,1.3373,-25.2709,6457.2796,576.2720,5.0706,307.9872,39.7548,1.1390,3.7165,1.5090,-1.0580,-1.2578,0.8741,-0.2208,-0.1368,0.9948,-0.5518,-0.6357,0.9319,-0.1739,-0.1591,0.9815,0.3069,0.5190,1.0693
std,191918.1941,0.5644,6.6882,16.7384,3.2111,3.7335,8.4004,15.9498,20.0470,54.3550,4.1897,7.4005,14.1642,11.0824,9.1005,14.8928,61.8387,5.3624,1.9953,1.2861,1.9733,19.1018,15.6818,0.4493,113.8532,2.4033,6624.4536,3831.5068,341.3178,7.6676,1220.5114,153.7794,3.6517,10.4069,1.5374,1.7790,2.6326,0.3006,1.5090,2.3064,0.2995,1.4596,2.0868,0.2671,0.9774,1.4178,0.1817,0.8046,1.2497,0.1633
min,2970.0000,-1.0600,27.4400,-70.2100,-14.2177,-26.6700,-51.0000,-71.0200,-70.8300,-85.8000,-18.6800,-44.7900,-66.3100,-68.4700,-5.0800,-95.7000,-5.0800,0.0200,0.0298,0.0100,0.0100,0.4119,0.7541,0.2800,2.3300,-15.6500,-186222.6781,-15.8745,0.0000,-32.7335,-14.6361,0.0000,-12.9965,-10.5329,0.0760,-9.4915,-8.6705,0.1978,-5.3699,-5.5677,0.3911,-4.5891,-5.2252,0.4477,-3.0179,-3.6310,0.5607,-2.0749,-2.6061,0.6740
25%,8862.5000,0.6500,45.3700,-13.3975,0.0335,-0.3175,-10.6000,-17.1425,-16.4275,-7.9925,-1.3400,-7.7650,-12.6075,-20.9300,9.5225,-29.8900,11.1300,1.0500,2.9633,1.9825,2.9600,1.4182,1.5200,0.7600,34.0500,0.0425,10.4979,324.5234,29.9108,1.5709,2.6376,1.2864,-1.4362,-1.8816,0.7845,-2.2272,-2.9840,0.6804,-1.1322,-1.5251,0.8122,-1.5940,-2.1542,0.7383,-0.8280,-1.1320,0.8557,-0.2052,-0.2919,0.9629
50%,17660.0000,0.9700,48.7750,-4.0400,1.5443,1.0450,-5.7100,-7.8300,-7.0150,5.4850,0.1750,-3.7150,-4.6700,-12.8850,13.0150,-18.4650,17.1950,2.4000,3.7939,2.6100,3.7400,2.5790,2.7200,0.9300,63.3850,1.1100,25.6140,8757.8179,781.2084,4.3134,8.5159,2.0234,0.4500,0.6011,1.0732,-1.2193,-1.7535,0.8087,-0.4601,-0.6603,0.9201,-0.7303,-1.0387,0.8733,-0.2915,-0.4113,0.9495,0.3014,0.4189,1.0563
75%,47127.5000,1.2400,53.6950,4.3775,3.0729,2.9100,-0.7125,1.8150,2.2750,21.8125,2.4475,0.4150,2.6675,-5.8975,17.5475,-9.3650,32.1850,4.8675,5.1458,3.3775,5.0175,5.9585,5.7475,1.1800,114.7475,2.2800,55.7057,8757.8179,781.2084,7.9928,22.9620,3.7346,3.1605,5.4857,1.6711,-0.0572,-0.0956,0.9900,0.5663,0.8556,1.1109,0.3374,0.4876,1.0641,0.4447,0.6512,1.0867,0.7725,1.1416,1.1492
max,3073520.0000,5.0300,88.6300,95.2500,35.0264,34.9700,60.6100,53.4000,134.6300,972.7300,52.6300,61.2100,81.3500,20.6600,113.7000,2.4900,1482.3800,44.5900,19.6307,10.3400,18.2800,361.6139,313.7900,5.5200,1049.5900,28.7800,77066.5880,8757.8179,781.2084,83.4628,5588.4762,705.0818,15.5952,166.2095,32.4106,6.9495,17.2926,3.1011,5.8002,12.3225,2.5868,4.9033,10.5589,2.5142,3.0444,5.0290,1.7018,3.6166,7.2274,1.7838


In [5]:
import pandas as pd
import os
import datetime

# --- Configuration ---
STATS_HISTORY_FILE = 'market_daily_summary_stats_history.csv' # Where to store the history

def append_daily_stats(df_daily_data, date_str, filename=STATS_HISTORY_FILE):
    """
    Calculates descriptive stats for df_daily_data, flattens them,
    and appends/updates them as a row in a historical CSV file.

    Args:
        df_daily_data (pd.DataFrame): DataFrame containing the raw data for ALL stocks for ONE specific day.
        date_str (str): The date for this data in 'YYYY-MM-DD' format.
        filename (str): The path to the CSV file storing the historical summary stats.
    """
    print(f"Processing stats for date: {date_str}")

    # 1. Calculate descriptive statistics for the input day's data
    try:
        daily_stats_raw = df_daily_data.describe()
        print("Successfully calculated describe() stats.")
    except Exception as e:
        print(f"Error calculating describe() stats: {e}")
        return # Stop if calculation fails

    # 2. Flatten the describe() output into a single row DataFrame
    try:
        # Unstack turns the stats (mean, std, etc.) into columns alongside the original metrics
        stats_flat = daily_stats_raw.unstack().to_frame().T
        # Create clearer column names like 'Metric_Statistic' (e.g., 'RSI_mean', 'SMA50 %_50%')
        stats_flat.columns = ['_'.join(col).strip() for col in stats_flat.columns.values]
        # Add the date to this row
        stats_flat['Date'] = pd.to_datetime(date_str)
        # Set the Date as the index for this single row
        stats_flat = stats_flat.set_index('Date')
        print("Successfully flattened stats into a single row.")
        # print(stats_flat) # Optional: print the flattened stats for verification

    except Exception as e:
        print(f"Error flattening the stats DataFrame: {e}")
        return # Stop if flattening fails

    # 3. Load existing history, append/update, and save
    try:
        df_summary_history = pd.DataFrame() # Initialize empty
        if os.path.exists(filename):
            print(f"Loading existing history from: {filename}")
            df_summary_history = pd.read_csv(filename, index_col='Date', parse_dates=True)

        # Check if the date already exists - update if it does, append if it doesn't
        current_date = pd.to_datetime(date_str)
        if current_date in df_summary_history.index:
            print(f"Updating existing entry for {date_str}")
            # Update existing row(s) - combines columns, replacing old values for this date
            df_summary_history.update(stats_flat)
            # Alternative: df_summary_history.loc[current_date] = stats_flat.iloc[0] # Direct overwrite
        else:
            print(f"Appending new entry for {date_str}")
            df_summary_history = pd.concat([df_summary_history, stats_flat])

        # Ensure the history is sorted by date
        df_summary_history = df_summary_history.sort_index()

        # Save the updated history back to CSV
        df_summary_history.to_csv(filename, index=True) # index=True saves the Date index
        print(f"Successfully saved updated stats history to: {filename}")

    except Exception as e:
        print(f"Error loading/saving the stats history file '{filename}': {e}")



In [6]:
# Simulate processing for a couple of days
print("-" * 20, "Day 1", "-" * 20)
append_daily_stats(df_data, date_str)

-------------------- Day 1 --------------------
Processing stats for date: 2025-04-23
Successfully calculated describe() stats.
Successfully flattened stats into a single row.
Loading existing history from: market_daily_summary_stats_history.csv
Appending new entry for 2025-04-23
Successfully saved updated stats history to: market_daily_summary_stats_history.csv


In [7]:
print(f'date_str: {date_str}')
df_data.describe()

date_str: 2025-04-23


,"MktCap AUM, M",Beta,RSI,Perf YTD %,Perf 3D %,Perf Week %,Perf Month %,Perf Quart %,Perf Half %,Perf Year %,SMA20 %,SMA50 %,SMA200 %,50D High %,50D Low %,52W High %,52W Low %,ATR,ATR/Price %,Volatility W %,Volatility M %,"Volume, M","Avg Volume, M",Rel Volume,Price,Change %,Sharpe 3d,Sortino 3d,Omega 3d,Sharpe 5d,Sortino 5d,Omega 5d,Sharpe 10d,Sortino 10d,Omega 10d,Sharpe 15d,Sortino 15d,Omega 15d,Sharpe 30d,Sortino 30d,Omega 30d,Sharpe 60d,Sortino 60d,Omega 60d,Sharpe 120d,Sortino 120d,Omega 120d,Sharpe 250d,Sortino 250d,Omega 250d
count,1202.0000,1202.0000,1202.0000,1202.0000,1202.0000,1202.0000,1202.0000,1202.0000,1202.0000,1202.0000,1202.0000,1202.0000,1202.0000,1202.0000,1202.0000,1202.0000,1202.0000,1202.0000,1202.0000,1202.0000,1202.0000,1202.0000,1202.0000,1202.0000,1202.0000,1202.0000,1202.0000,1202.0000,1202.0000,1202.0000,1202.0000,1202.0000,1202.0000,1202.0000,1202.0000,1202.0000,1202.0000,1202.0000,1202.0000,1202.0000,1202.0000,1202.0000,1202.0000,1202.0000,1202.0000,1202.0000,1202.0000,1202.0000,1202.0000,1202.0000
mean,58322.0715,0.9930,49.7131,-3.7685,1.8049,1.3888,-5.8134,-7.4467,-5.3862,12.3486,0.5539,-3.7544,-4.4720,-14.7292,14.5073,-20.8245,30.3075,4.1171,4.0901,2.7083,4.0342,7.2051,6.6729,1.0309,99.2806,1.3373,-25.2709,6457.2796,576.2720,5.0706,307.9872,39.7548,1.1390,3.7165,1.5090,-1.0580,-1.2578,0.8741,-0.2208,-0.1368,0.9948,-0.5518,-0.6357,0.9319,-0.1739,-0.1591,0.9815,0.3069,0.5190,1.0693
std,191918.1941,0.5644,6.6882,16.7384,3.2111,3.7335,8.4004,15.9498,20.0470,54.3550,4.1897,7.4005,14.1642,11.0824,9.1005,14.8928,61.8387,5.3624,1.9953,1.2861,1.9733,19.1018,15.6818,0.4493,113.8532,2.4033,6624.4536,3831.5068,341.3178,7.6676,1220.5114,153.7794,3.6517,10.4069,1.5374,1.7790,2.6326,0.3006,1.5090,2.3064,0.2995,1.4596,2.0868,0.2671,0.9774,1.4178,0.1817,0.8046,1.2497,0.1633
min,2970.0000,-1.0600,27.4400,-70.2100,-14.2177,-26.6700,-51.0000,-71.0200,-70.8300,-85.8000,-18.6800,-44.7900,-66.3100,-68.4700,-5.0800,-95.7000,-5.0800,0.0200,0.0298,0.0100,0.0100,0.4119,0.7541,0.2800,2.3300,-15.6500,-186222.6781,-15.8745,0.0000,-32.7335,-14.6361,0.0000,-12.9965,-10.5329,0.0760,-9.4915,-8.6705,0.1978,-5.3699,-5.5677,0.3911,-4.5891,-5.2252,0.4477,-3.0179,-3.6310,0.5607,-2.0749,-2.6061,0.6740
25%,8862.5000,0.6500,45.3700,-13.3975,0.0335,-0.3175,-10.6000,-17.1425,-16.4275,-7.9925,-1.3400,-7.7650,-12.6075,-20.9300,9.5225,-29.8900,11.1300,1.0500,2.9633,1.9825,2.9600,1.4182,1.5200,0.7600,34.0500,0.0425,10.4979,324.5234,29.9108,1.5709,2.6376,1.2864,-1.4362,-1.8816,0.7845,-2.2272,-2.9840,0.6804,-1.1322,-1.5251,0.8122,-1.5940,-2.1542,0.7383,-0.8280,-1.1320,0.8557,-0.2052,-0.2919,0.9629
50%,17660.0000,0.9700,48.7750,-4.0400,1.5443,1.0450,-5.7100,-7.8300,-7.0150,5.4850,0.1750,-3.7150,-4.6700,-12.8850,13.0150,-18.4650,17.1950,2.4000,3.7939,2.6100,3.7400,2.5790,2.7200,0.9300,63.3850,1.1100,25.6140,8757.8179,781.2084,4.3134,8.5159,2.0234,0.4500,0.6011,1.0732,-1.2193,-1.7535,0.8087,-0.4601,-0.6603,0.9201,-0.7303,-1.0387,0.8733,-0.2915,-0.4113,0.9495,0.3014,0.4189,1.0563
75%,47127.5000,1.2400,53.6950,4.3775,3.0729,2.9100,-0.7125,1.8150,2.2750,21.8125,2.4475,0.4150,2.6675,-5.8975,17.5475,-9.3650,32.1850,4.8675,5.1458,3.3775,5.0175,5.9585,5.7475,1.1800,114.7475,2.2800,55.7057,8757.8179,781.2084,7.9928,22.9620,3.7346,3.1605,5.4857,1.6711,-0.0572,-0.0956,0.9900,0.5663,0.8556,1.1109,0.3374,0.4876,1.0641,0.4447,0.6512,1.0867,0.7725,1.1416,1.1492
max,3073520.0000,5.0300,88.6300,95.2500,35.0264,34.9700,60.6100,53.4000,134.6300,972.7300,52.6300,61.2100,81.3500,20.6600,113.7000,2.4900,1482.3800,44.5900,19.6307,10.3400,18.2800,361.6139,313.7900,5.5200,1049.5900,28.7800,77066.5880,8757.8179,781.2084,83.4628,5588.4762,705.0818,15.5952,166.2095,32.4106,6.9495,17.2926,3.1011,5.8002,12.3225,2.5868,4.9033,10.5589,2.5142,3.0444,5.0290,1.7018,3.6166,7.2274,1.7838


In [8]:
print(f'date_str: {date_str}')
df_data.head()

date_str: 2025-04-23


,Company,Info,"MktCap AUM, M",Beta,RSI,Perf YTD %,Perf 3D %,Perf Week %,Perf Month %,Perf Quart %,Perf Half %,Perf Year %,SMA20 %,SMA50 %,SMA200 %,50D High %,50D Low %,52W High %,52W Low %,ATR,ATR/Price %,Volatility W %,Volatility M %,"Volume, M","Avg Volume, M",Rel Volume,Price,Change %,Sharpe 3d,Sortino 3d,Omega 3d,Sharpe 5d,Sortino 5d,Omega 5d,Sharpe 10d,Sortino 10d,Omega 10d,Sharpe 15d,Sortino 15d,Omega 15d,Sharpe 30d,Sortino 30d,Omega 30d,Sharpe 60d,Sortino 60d,Omega 60d,Sharpe 120d,Sortino 120d,Omega 120d,Sharpe 250d,Sortino 250d,Omega 250d
Ticker,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,
AAPL,Apple Inc,"Technology, Consumer Electronics",3073520.0000,1.2900,49.0400,-18.3000,3.8684,1.2200,-7.3100,-8.5900,-12.9400,24.0000,0.8200,-7.0000,-10.1900,-18.1600,20.9100,-21.3400,24.0600,9.9900,4.8827,2.6900,4.8400,52.0936,61.3900,0.8500,204.6000,2.4300,66.9795,8757.8179,781.2084,8.9293,21.2395,3.6759,1.7656,2.6850,1.3000,-1.2684,-1.9802,0.8076,-0.4748,-0.7227,0.9142,-1.0764,-1.5500,0.8079,-0.6064,-0.8638,0.8847,0.6438,0.9567,1.1350
MSFT,Microsoft Corporation,"Technology, Software - Infrastructure",2783210.0000,0.9800,47.3400,-11.1800,1.7973,-2.9400,-4.7500,-16.0900,-10.4700,-6.2000,-0.4000,-3.5200,-10.0200,-10.7100,8.5800,-20.0600,8.5800,12.8700,3.4376,2.4800,3.3600,20.2058,25.2100,0.8000,374.3900,2.0600,582.6925,8757.8179,781.2084,1.3341,2.3249,1.2221,-3.4440,-4.3079,0.5914,-0.4842,-0.9400,0.9150,-0.3997,-0.7150,0.9265,-2.1398,-3.0785,0.6614,-0.9786,-1.3746,0.8308,-0.3687,-0.5088,0.9352
NVDA,NVIDIA Corp,"Technology, Semiconductors",2506120.0000,2.1100,44.5900,-23.5200,1.2021,-8.4600,-15.4000,-30.1600,-25.5700,34.7900,-2.8300,-11.5900,-18.3400,-28.4000,18.5800,-32.9300,31.3000,7.1100,6.9224,4.1600,6.3600,245.3014,313.7900,0.7800,102.7100,3.8600,36.2350,8757.8179,781.2084,-1.5428,-2.2780,0.7922,-4.4274,-5.0789,0.5044,-0.7381,-1.2607,0.8786,-0.9640,-1.5560,0.8356,-1.0929,-1.5654,0.8281,-0.8050,-1.0933,0.8679,0.6615,0.9583,1.1193
AMZN,Amazon.com Inc,"Consumer Cyclical, Internet Retail",1916590.0000,1.3100,46.2300,-17.6800,4.6289,0.5600,-11.1500,-23.1500,-4.4400,3.4200,-1.2000,-8.9100,-9.4000,-22.7900,11.9100,-25.5300,19.1200,9.7100,5.3765,3.2800,5.0400,61.8203,48.8300,1.2700,180.6000,4.2800,111.2769,8757.8179,781.2084,4.1218,8.9966,1.8993,-2.9677,-4.0059,0.6450,-1.5029,-2.2917,0.7822,-1.1801,-1.7341,0.8164,-2.4154,-3.2714,0.6553,-0.2043,-0.2953,0.9656,0.1170,0.1650,1.0206
GOOGL,Alphabet Inc,"Communication Services, Internet Content & Inf...",1907420.0000,1.0000,47.6800,-17.9300,2.7719,-0.6100,-7.3500,-21.6900,-4.9400,0.8200,0.8400,-5.9300,-9.9800,-17.4500,10.5500,-24.9700,10.5500,5.9900,3.8558,2.8600,3.7700,30.3598,35.0200,0.8700,155.3500,2.5600,4878.4214,8757.8179,781.2084,2.0699,3.9174,1.3587,-1.4315,-2.0259,0.8181,-0.1410,-0.2536,0.9772,-1.2256,-1.8601,0.8185,-2.2889,-3.0174,0.6820,-0.4243,-0.6060,0.9316,-0.0291,-0.0418,0.9949
